In [51]:
import pandas as pd
import numpy as np
import torch
import pickle
from torch.utils.data import TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from util import preprocess_data, compute_metrics, foresee

SEED = 2137
PATH_FOR_MODEL = '../models/auto_days_name'
EPOCHS = 10

data = pd.read_csv('../data/recruitment_data_modified_python.csv',
                   encoding='utf-8',
                   sep=',',
                   on_bad_lines='warn',
                   quotechar='"',
                   doublequote=True,
                   names=['URL', 'Voivodeship', 'Scrap_time', 'Name', 'Price', 'Brand', 'Condition', 'Offer_from', 'Type', 'Description', 'Added_at', 'Views', 'User_since'],
                   skiprows=1)

preprocessed_data = preprocess_data(data)

Skipping line 346: expected 13 fields, saw 14
Skipping line 367: expected 13 fields, saw 15
Skipping line 466: expected 13 fields, saw 19
Skipping line 467: expected 13 fields, saw 19
Skipping line 494: expected 13 fields, saw 15
Skipping line 615: expected 13 fields, saw 15
Skipping line 616: expected 13 fields, saw 15
Skipping line 617: expected 13 fields, saw 15
Skipping line 618: expected 13 fields, saw 15
Skipping line 619: expected 13 fields, saw 15
Skipping line 620: expected 13 fields, saw 15
Skipping line 712: expected 13 fields, saw 14
Skipping line 713: expected 13 fields, saw 14
Skipping line 739: expected 13 fields, saw 14
Skipping line 747: expected 13 fields, saw 16
Skipping line 867: expected 13 fields, saw 15
Skipping line 956: expected 13 fields, saw 14
Skipping line 1028: expected 13 fields, saw 14
Skipping line 1234: expected 13 fields, saw 15
Skipping line 1282: expected 13 fields, saw 17
Skipping line 1326: expected 13 fields, saw 14
Skipping line 1327: expected 1

In [2]:
preprocessed_data

,Price,Days_passed_name,Days_passed_name_desc
0,2799.0,52 days iphone 11 64 jak nowy 95% gwarancja wy...,52 days iphone 11 64 jak nowy 95% gwarancja wy...
1,2700.0,"51 days iphone 11 64 gb czarny, idealny z gwar...","51 days iphone 11 64 gb czarny, idealny z gwar..."
2,2899.0,51 days jak nowy apple iphone 11 256gbgb white...,51 days jak nowy apple iphone 11 256gbgb white...
3,2500.0,51 days apple iphone 11 biały 64gb - jak nowy ...,51 days apple iphone 11 biały 64gb - jak nowy ...
4,2150.0,51 days iphone 11 64 gb + gwarancja,"51 days iphone 11 64 gb + gwarancja witam, mam..."
...,...,...,...
2667,2299.0,51 days iphone 11 black 64gb,51 days iphone 11 black 64gb sprzedam iphone 1...
2668,1900.0,51 days i phone 11 64 gb cena tylko dzis,51 days i phone 11 64 gb cena tylko dzis cena ...
2669,2800.0,"51 days iphone 11 128 gb gwarancja , 100% bat...","51 days iphone 11 128 gb gwarancja , 100% bat..."
2670,1650.0,50 days iphone 11 white 64gb,50 days iphone 11 white 64gb na sprzedaż posia...


In [48]:
preprocessed_data['Days_passed_name'].tolist()

['52 days iphone 11 64 jak nowy 95% gwarancja wyświetlacz',
 '51 days iphone 11 64 gb czarny, idealny z gwarancją. wymiana',
 '51 days jak nowy apple iphone 11 256gbgb white gwarancja',
 '51 days apple iphone 11 biały 64gb - jak nowy gwarancja paragon 4xetui',
 '51 days iphone 11 64 gb + gwarancja',
 '51 days iphone 11 64 gb + gwarancja',
 '51 days iphone 11 w bardzo dobrym stanie',
 '51 days iphone 11 * idealny stan * 100% bateria * gwarancja * zamiana *',
 '51 days iphone 11 128 gb stan idealny etui gratis',
 '52 days iphone 11 64 gb prawie nowy super zestaw',
 '52 days iphone 11 czarny 128 gb',
 '51 days oryginalny | apple iphone 11 64/128gb | różne kolory | rok gwarancji',
 '51 days sprzedam iphone 11 128gb red 23msc gwarancji',
 '52 days iphone 11 64gb fioletowy perfekcyjny purple',
 '51 days apple iphone 11 64gb red poznań długa 14',
 '51 days iphone 11 ideał gwarancja 256gb',
 '51 days telefon iphone se (2020) gw do 11.09.2021',
 '51 days iphone  11 ( rezerwacja )',
 '51 days ip

In [52]:
model = AutoModelForSequenceClassification.from_pretrained('dkleczek/bert-base-polish-uncased-v1', num_labels = 1).to("cpu")

loading configuration file config.json from cache at C:\Users\cubix/.cache\huggingface\hub\models--dkleczek--bert-base-polish-uncased-v1\snapshots\62be9821055981deafb23f217b68cc41f38cdb76\config.json
Model config BertConfig {
  "_name_or_path": "dkleczek/bert-base-polish-uncased-v1",
  "architectures": [
    "BertForMaskedLM",
    "BertForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 60000
}

loading weights file pytorch

In [54]:
tokenizer = AutoTokenizer.from_pretrained('dkleczek/bert-base-polish-uncased-v1')

scaler = StandardScaler()
y_dataset = scaler.fit_transform(np.asarray(preprocessed_data['Price']).reshape(-1,1))

X_train, X_test, y_train, y_test = train_test_split(preprocessed_data['Days_passed_name'].tolist(), y_dataset, train_size=0.8)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size=0.8)

train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=50)
valid_encodings = tokenizer(X_val, truncation=True, padding=True, max_length=50)
test_encodings = tokenizer(X_test, truncation=True, padding=True, max_length=50)

class MakeTorchData(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        item["labels"] = float(item["labels"])
        return item

    def __len__(self):
        return len(self.labels)

# convert our tokenized data into a torch Dataset
train_dataset = MakeTorchData(train_encodings, y_train.ravel())
valid_dataset = MakeTorchData(valid_encodings, y_val.ravel())
test_dataset = MakeTorchData(test_encodings, y_test.ravel())

loading configuration file config.json from cache at C:\Users\cubix/.cache\huggingface\hub\models--dkleczek--bert-base-polish-uncased-v1\snapshots\62be9821055981deafb23f217b68cc41f38cdb76\config.json
Model config BertConfig {
  "_name_or_path": "dkleczek/bert-base-polish-uncased-v1",
  "architectures": [
    "BertForMaskedLM",
    "BertForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 60000
}

loading file vocab.txt from cache at C:\Users\cubix/.cache\huggingface\hub\models--dkleczek--bert-base-pol

In [6]:
# Specify the arguments for the trainer
training_args = TrainingArguments(
    output_dir = PATH_FOR_MODEL + '/checkpoints',
    num_train_epochs = EPOCHS,
    per_device_train_batch_size = 64,
    per_device_eval_batch_size = 20,
    weight_decay = 0.01,
    learning_rate = 2e-5,
    logging_dir = '../logs',
    save_total_limit = 10,
    load_best_model_at_end = True,
    metric_for_best_model = 'rmse',
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
)

# Call the Trainer
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = valid_dataset,
    compute_metrics = compute_metrics,
)

# Train the model
trainer.train()

# Call the summary
trainer.evaluate()

E:\nlp2\venv\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1709
  Num Epochs = 5
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 135
  Number of trainable parameters = 132122113


Epoch,Training Loss,Validation Loss


***** Running Evaluation *****
  Num examples = 428
  Batch size = 20
Saving model checkpoint to ../results\checkpoint-27
Configuration saved in ../results\checkpoint-27\config.json
Model weights saved in ../results\checkpoint-27\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 428
  Batch size = 20
Saving model checkpoint to ../results\checkpoint-54
Configuration saved in ../results\checkpoint-54\config.json
Model weights saved in ../results\checkpoint-54\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 428
  Batch size = 20
Saving model checkpoint to ../results\checkpoint-81
Configuration saved in ../results\checkpoint-81\config.json
Model weights saved in ../results\checkpoint-81\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 428
  Batch size = 20
Saving model checkpoint to ../results\checkpoint-108
Configuration saved in ../results\checkpoint-108\config.json
Model weights saved in ../results\checkpoint-108\pytorch_model.bin
*****

{'eval_loss': 0.5495059490203857,
 'eval_mse': 0.5495059490203857,
 'eval_rmse': 0.7412866950035095,
 'eval_mae': 0.4915722906589508,
 'eval_r2': 0.39628914202738397,
 'eval_runtime': 36.6704,
 'eval_samples_per_second': 11.672,
 'eval_steps_per_second': 0.6,
 'epoch': 5.0}

In [50]:
trainer.save_model(PATH_FOR_MODEL)

Saving model checkpoint to ../models/trainer_auto_model
Configuration saved in ../models/trainer_auto_model\config.json
Model weights saved in ../models/trainer_auto_model\pytorch_model.bin


In [7]:
predictions = trainer.predict(test_dataset)

***** Running Prediction *****
  Num examples = 535
  Batch size = 20


In [8]:
inversed_predictions = scaler.inverse_transform(np.asarray(predictions[0]).reshape(-1,1))
inversed_test = scaler.inverse_transform(np.asarray(predictions[1]).reshape(-1, 1))
mean_squared_error(inversed_test, inversed_predictions, squared=False)

413.315

In [9]:
inversed_test - inversed_predictions

array([[-7.25620117e+02],
       [ 9.91364746e+01],
       [-9.25512695e+01],
       [ 3.61143066e+02],
       [-1.23446533e+02],
       [-9.38164062e+01],
       [-6.15995850e+02],
       [-4.29147949e+02],
       [-2.87149170e+02],
       [ 4.26442871e+01],
       [-5.22445312e+02],
       [-4.75187988e+01],
       [-4.43587646e+02],
       [-1.17280273e+02],
       [ 4.09817383e+02],
       [-4.39255371e+01],
       [-1.50767334e+02],
       [-2.50756836e+00],
       [-1.02550732e+03],
       [-8.35827637e+01],
       [ 2.61540283e+02],
       [ 3.90700684e+01],
       [-4.15114746e+01],
       [-1.11669922e+00],
       [ 7.17651367e+00],
       [ 2.40712158e+02],
       [-7.89628906e+01],
       [ 4.34648438e+01],
       [-5.98999023e+01],
       [ 6.03100586e+01],
       [ 5.08358643e+02],
       [ 1.46776367e+02],
       [ 1.11430908e+02],
       [-7.99707520e+02],
       [-1.09870605e+01],
       [ 1.15310303e+02],
       [-2.29973145e+02],
       [-5.49350830e+02],
       [ 1.5

In [16]:
extrapolated_x = []
extrapolated_y = []
for i in range(51, 85):
    extrapolated_x.append(str(i) + ' days' + ' iphone 11')
    extrapolated_y.append(0)

In [17]:
extrapolated_x

['51 days iphone 11',
 '52 days iphone 11',
 '53 days iphone 11',
 '54 days iphone 11',
 '55 days iphone 11',
 '56 days iphone 11',
 '57 days iphone 11',
 '58 days iphone 11',
 '59 days iphone 11',
 '60 days iphone 11',
 '61 days iphone 11',
 '62 days iphone 11',
 '63 days iphone 11',
 '64 days iphone 11',
 '65 days iphone 11',
 '66 days iphone 11',
 '67 days iphone 11',
 '68 days iphone 11',
 '69 days iphone 11',
 '70 days iphone 11',
 '71 days iphone 11',
 '72 days iphone 11',
 '73 days iphone 11',
 '74 days iphone 11',
 '75 days iphone 11',
 '76 days iphone 11',
 '77 days iphone 11',
 '78 days iphone 11',
 '79 days iphone 11',
 '80 days iphone 11',
 '81 days iphone 11',
 '82 days iphone 11',
 '83 days iphone 11',
 '84 days iphone 11']

In [12]:
extrapolated_tokens = tokenizer(extrapolated_x, truncation=True, padding=True, max_length=50)

extrapolated_dataset = MakeTorchData(extrapolated_tokens, np.asarray(extrapolated_y).ravel())

In [13]:
extrapolated_predictions = trainer.predict(extrapolated_dataset)

***** Running Prediction *****
  Num examples = 34
  Batch size = 20


In [15]:
inversed_extrapolated = scaler.inverse_transform(np.asarray(extrapolated_predictions[0]).reshape(-1, 1))
type(inversed_extrapolated)

numpy.ndarray

In [39]:
# def foresee(model: Trainer,
#             scaler: StandardScaler,
#             tokenizer: AutoTokenizer,
#             text: str = 'iphone 11',
#             days: tuple = (1, 90)) -> np.ndarray:
#
#     extrapolated_x = []
#     extrapolated_y = []
#     for i in range(*days):
#         extrapolated_x.append(str(i) + ' days ' + text)
#         extrapolated_y.append(0)
#
#     extrapolated_tokens = tokenizer(extrapolated_x, truncation=True, padding=True, max_length=50)
#     extrapolated_dataset = MakeTorchData(extrapolated_tokens, np.asarray(extrapolated_y).ravel())
#     extrapolated_predictions = model.predict(extrapolated_dataset)
#     inversed_extrapolated = scaler.inverse_transform(np.asarray(extrapolated_predictions[0]).reshape(-1, 1))
#
#     return inversed_extrapolated

In [44]:
foresee(trainer, scaler, tokenizer, 'iphone 11 64')

***** Running Prediction *****
  Num examples = 89
  Batch size = 20


array([[2403.2437],
       [2442.9517],
       [2437.8252],
       [2451.3066],
       [2422.7432],
       [2444.0386],
       [2417.8943],
       [2446.6145],
       [2441.293 ],
       [2441.623 ],
       [2434.404 ],
       [2440.8982],
       [2429.606 ],
       [2398.373 ],
       [2417.2644],
       [2418.433 ],
       [2416.4912],
       [2433.04  ],
       [2404.2578],
       [2401.4277],
       [2399.366 ],
       [2426.9976],
       [2438.076 ],
       [2445.7385],
       [2435.1216],
       [2401.7002],
       [2387.809 ],
       [2374.431 ],
       [2380.53  ],
       [2394.7136],
       [2412.0156],
       [2413.5825],
       [2421.7454],
       [2427.422 ],
       [2429.918 ],
       [2424.5562],
       [2428.7966],
       [2445.3   ],
       [2449.1538],
       [2415.9023],
       [2404.0432],
       [2408.225 ],
       [2412.148 ],
       [2411.6958],
       [2412.03  ],
       [2386.2896],
       [2370.4631],
       [2418.9194],
       [2415.4258],
       [2402.1667],


In [ ]:
pickle.dump(scaler, open(PATH_FOR_MODEL + '/scaler.pkl','wb'))
# scaler = pickle.load(open('../results/scaler_auto_model.pkl','rb'))